In [ ]:
import pandas as pd
from seeq import spy
from seeq.spy.assets import Asset, ItemGroup


In [ ]:
WORKBOOK_NAME = 'F1 Telemetry Testing'
DATASOURCE_NAME = 'F1 Telemetry Playground'
SESSION_CONDITION_NAME = 'F1 Sessions'


In [ ]:
def get_f1_items():
    scoped_workbook_id = None
    if WORKBOOK_NAME != None:
        scoped_workbook = spy.search({'Name':WORKBOOK_NAME, 'Type':'Workbook'})
        scoped_workbook_id = scoped_workbook['ID'][0]
    
    telemetry_items = spy.search({'Name':'20??.*.*.*','Datasource Name':DATASOURCE_NAME, 'Scoped To':scoped_workbook_id})
    return telemetry_items

In [ ]:
class F1_Car(Asset):
    
    @Asset.Attribute()
    def Sectors(self, metadata):
        return metadata[metadata['Name'].str.endswith('Sectors')]
    
    @Asset.Attribute()
    def X_pos(self, metadata):
        return metadata[metadata['Name'].str.endswith('X')]

    @Asset.Attribute()
    def Y_pos(self, metadata):
        return metadata[metadata['Name'].str.endswith('Y')]

    @Asset.Attribute()
    def Z_pos(self, metadata):
        return metadata[metadata['Name'].str.endswith('Z')]

    @Asset.Attribute()
    def RPM(self, metadata):
        return metadata[metadata['Name'].str.endswith('rpm')]

    @Asset.Attribute()
    def Throttle(self, metadata):
        return metadata[metadata['Name'].str.endswith('throttle')]

    @Asset.Attribute()
    def Brake(self, metadata):
        return metadata[metadata['Name'].str.endswith('brake')]

    @Asset.Attribute()
    def Gear(self, metadata):
        return metadata[metadata['Name'].str.endswith('n_gear')]

    @Asset.Attribute()
    def TimestampPOS(self, metadata):
        return metadata[metadata['Name'].str.endswith('timestamp_pos')]

    @Asset.Attribute()
    def Speed(self, metadata):
        return metadata[metadata['Name'].str.endswith('speed')]

    @Asset.Attribute()
    def LapNumber(self, metadata):
        return metadata[metadata['Name'].str.endswith('lap_number')]

    @Asset.Attribute()
    def DRS(self, metadata):
        return metadata[metadata['Name'].str.endswith('drs')]

    @Asset.Attribute()
    def SessionKeyPOS(self, metadata):
        return metadata[metadata['Name'].str.endswith('SessionKey_pos')]

    @Asset.Attribute()
    def Status(self, metadata):
        return metadata[metadata['Name'].str.endswith('Status')]

#####  Calculations

    @Asset.Attribute()
    def Laps(self, metadata):
        return {
            'Type': 'Condition',
            
            # This formula will create a capsule for every change in the lap number value
            'Formula': "$h.toCondition().renameProperty('value','lap number')",
            
            'Formula Parameters': {
                # We can reference the base class lap number
                '$h': self.LapNumber(),
            }
        }
            
    @Asset.Attribute()
    def Sector_1(self, metadata):
        return {
            'Type': 'Condition',
            
            # This formula will create a new condition that filters out only sector 1
            'Formula': "$condition.keep('Sector', isEqualTo(1))",
            
            'Formula Parameters': {
                '$condition': self.Sectors(),
            }
        }

    @Asset.Attribute()
    def Sector_2(self, metadata):
        return {
            'Type': 'Condition',
            
            'Formula': "$condition.keep('Sector', isEqualTo(2))",
            
            'Formula Parameters': {
                '$condition': self.Sectors(),
            }
        }

    @Asset.Attribute()
    def Sector_3(self, metadata):
        return {
            'Type': 'Condition',
            
            'Formula': "$condition.keep('Sector', isEqualTo(3))",
            
            'Formula Parameters': {
                '$condition': self.Sectors(),
            }
        }
    

In [ ]:
telemetry_items = get_f1_items()
telemetry_items[['Season', 'Team', 'Car Number', 'Tag Name']] = telemetry_items['Name'].str.split('.', expand=True)
telemetry_items['Build Asset'] = telemetry_items['Car Number'] 
telemetry_items ['Build Path'] = 'F1 Telemetry >> ' + telemetry_items['Season'] + ' >> ' + telemetry_items['Team']
telemetry_items

In [ ]:
build_df = spy.assets.build(F1_Car, telemetry_items)
build_df

In [ ]:
spy.push(metadata=build_df, workbook=WORKBOOK_NAME)